In [1]:
from transformers import AutoModelForCausalLM, pipeline
import torch
import time
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain

inicio = time.time()

# Number of times to execute the code
num_executions = 10

# Output file path
output_file_path = "output_results_ON_3R_v4.txt"

# Loop for multiple executions
for i in range(num_executions):
    print(f"Execution {i+1}/{num_executions}")
    
    context = f"""
    Requirement1: Once the account is locked, the system sends an account lock notification email to the account’s owner.
    Requirement2: Once an account is locked, the system would also send a SMS message to the account’s owner to notify him about the situation owner.
    Requirement3: If a user has already received a notification via email, he will not receive the same notification via SMS.

    """

    template = """   

    I will present various software requirements in the context delimited by angle brackets. These proposed requirements may contradict each other. \

    A contradiction in software requirements refers to a scenario where two or more specified requirements are mutually exclusive or incompatible, \
    preventing their simultaneous fulfillment or coexistence within the system.
    
    Provide a unique response with the following structure: 
    
    Identify contradiction: Respond with a simple 'yes' or 'no'.
    Argument: Present a concise explanation, limited to 50 words, to justify your response.
    
    If you don't have the information then give response \"I don't know\".

    Context:
    <{context}>

    Question: {question}

      Response:
      """

    def build_qa_chain():
        torch.cuda.empty_cache()
        model_name = "databricks/dolly-v2-3b"

        instruct_pipeline = pipeline(model=model_name, 
                                 torch_dtype=torch.bfloat16,
                                 device_map="auto",
                                 return_full_text=True,
                                 max_new_tokens=256,
                                 top_p=0.95,
                                 top_k=50,
                                 trust_remote_code=True 
                                # model_kwargs={'load_in_8bit': True}
                                )

        model = AutoModelForCausalLM.from_pretrained(model_name, 
                                                     device_map="auto",
                                                     torch_dtype=torch.float16)

        return instruct_pipeline, model

    instruct_pipeline, model = build_qa_chain()


    question = "Do any of the provided requirements contradict each other, or are there conflicting conditions that prevent their simultaneous fulfillment within the system?"


    response = instruct_pipeline(template.format(context=context, question=question))
    
    # Extract the response from the output
    response_text = response[0]['generated_text'].split('Response:')[1].strip()
    print(response_text)

     # Print and save the output to the file
    with open(output_file_path, "a") as output_file:
        output_file.write(f"Execution {i+1}/{num_executions}:\n")
        output_file.write(response_text)
        output_file.write("\n\n")

print("All executions completed.")

fin = time.time()
print((fin - inicio)/60)

/Users/carlotaruiz/anaconda3/envs/gatenlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-14 12:09:28.139578: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Execution 1/10
Requirement 1 and 2: No. Both requirements are compatible with each other.

Requirement3: Yes. Since the condition is not fulfilled, a notification is not send.
Execution 2/10
No, there are no contradictions. All the requirements can be fulfilled simultaneously within the system.
Execution 3/10
No, there are no contradictory requirements.
Execution 4/10
Yes, there is a contradiction in the requirements. 

Both requirement 2 and 3 specify mutual exclusive conditions, prohibiting their simultaneous fulfillment or coexistence within the system.
Execution 5/10
None of the requirements contradict each other. The conditions mentioned above do not exist as both requirements 1 and 2 are fulfilled.
Execution 6/10
The requirements are not contradictory as they do not prohibit fulfillment. The email notification may be sent but also may not be sent. The SMS notification may be given but also may not be given. Hence, the requirements do not contradict.
Execution 7/10
No, there are n